In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
def pixelToBin(img, width):
    lst = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            #print(img[i][j],np.binary_repr(img[i][j] ,width=width))
            lst.append(np.binary_repr(img[i][j] ,width=width)) # width = no. of bits

    #print(lst)
    return lst

In [16]:
def bitPlaneSlicing(img, width):
    slicedList = []
    lst = pixelToBin(img, width)
    for planeNumber in range(width):
        #print(planeNumber, np.power(2,planeNumber))
        slicedList.append( (np.array([int(i[width -planeNumber - 1]) for i in lst],dtype = np.uint8) * np.power(2,planeNumber) ).reshape(img.shape[0],img.shape[1]) )
    
    return slicedList 

In [20]:
def get_pixel(img, center, x, y): 
      
    new_value = 0
      
    try: 
        # If local neighbourhood pixel  
        # value is greater than or equal 
        # to center pixel values then  
        # set it to 1 
        if img[x][y] >= center: 
            new_value = 1
              
    except: 
        # Exception is required when  
        # neighbourhood value of a center 
        # pixel value is null i.e. values 
        # present at boundaries. 
        pass
      
    return new_value 

In [ ]:
# Function for calculating LBP 
def lbp_calculated_pixel(img, line, col): 
   
    center = img[line][col] 
   
    val_ar = [] 
      
    # top_left 
    val_ar.append(get_pixel(img, center, line-1, col-1)) 
      
    # top 
    val_ar.append(get_pixel(img, center, line-1, col)) 
      
    # top_right 
    val_ar.append(get_pixel(img, center, line-1, col + 1)) 
      
    # right 
    val_ar.append(get_pixel(img, center, line, col + 1)) 
      
    # bottom_right 
    val_ar.append(get_pixel(img, center, line + 1, col + 1)) 
      
    # bottom 
    val_ar.append(get_pixel(img, center, line + 1, col)) 
      
    # bottom_left 
    val_ar.append(get_pixel(img, center, line + 1, col-1)) 
      
    # left 
    val_ar.append(get_pixel(img, center, line, col-1)) 
       
    # Now, we need to convert binary 
    # values to decimal 
    power_val = [1, 2, 4, 8, 16, 32, 64, 128] 
   
    val = 0
      
    for i in range(len(val_ar)): 
        val += val_ar[i] * np.power(2, i) 
          
    return val 

In [ ]:
def LBP(img):
    img_lbp = np.zeros((height, width), np.uint8) 
    for line in range(img.shape[0]):
        for col in range(img.shape[1]):
            img_lbp[line, col] = lbp_calculated_pixel(img, line, col)
    

In [17]:
img = cv2.imread("e.png")
#print(img)
#x = np.array([[6,7,6,6,7],[0,0,0,1,2],[1,1,1,2,3],[4,5,5,4,2],[6,6,6,7,7]],np.int32)
#360x480
resized_img = cv2.resize(img,(480,360))
#print(x.shape)
#z = bitPlaneSlicing(x,3)
#print(z)

b,g,r = cv2.split(resized_img)
#cv2.imshow('B-RGB',b)
#cv2.imshow('G-RGB',g)
#cv2.imshow('R-RGB',r)

In [19]:
clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
cl_img = clahe.apply(r)
#ret, otsu_cl_img = cv2.threshold(cl_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

slicedList = bitPlaneSlicing(cl_img,8)
#print(slicedList)
#cv2.imshow("Bit Plane "+str(i), )
finalr = cv2.hconcat([slicedList[7],slicedList[6],slicedList[5],slicedList[4]])
finalv = cv2.hconcat([slicedList[3],slicedList[2],slicedList[1],slicedList[0]])
final = cv2.vconcat([finalr,finalv])
cv2.imshow("final",final)
#cv2.imshow("8to5img",slicedList[7]+slicedList[6]+slicedList[5]+slicedList[4])
#cv2.imshow("4to1img",slicedList[3]+slicedList[2]+slicedList[1]+slicedList[0])
#cv2.imshow("CL_IMG",cl_img)
#cv2.imshow('B-RGB',b)
cv2.imshow('CLIMG',r)

# waits for user to press any key 
# (this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0)
# closing all open windows 
cv2.destroyAllWindows()